# Review Instructions

Please review the MSv4 `field_and_source_xds` schema and the XRADIO interface (`ps['MSv4_name'].VISIBILITY.field_and_source_xds`). Note that the PS (processing set) interface or the main_xds should not be reviewed.

The `field_and_source_xds` schema specification: https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1658760192

## Preparatory Material
Go over Xarray nomenclature and selection syntax:
- https://docs.xarray.dev/en/latest/user-guide/terminology.html
- https://docs.xarray.dev/en/latest/user-guide/indexing.html

MSv2 and CASA documentation:
- MSv2 schema: https://casacore.github.io/casacore-notes/229.pdf
- MSv3 schema: https://casacore.github.io/casacore-notes/264.pdf
- Ephemeris Data in CASA: https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data

## `field_and_source_xds` Schema
The FIELD, SOURCE, and EPHEMERIS tables in the MSv2 contain closely related information:
- **FIELD**: Field position for a source.
- **SOURCE**: Information about the source being observed (position, proper motion, etc.)
- **EPHEMERIS**: Ephemerides of the source.

These can be combined into a single dataset for MSv4 because it consists of a single field and consequently a single source[^1].

### Use Cases
The use cases considered during the design of the schema were:
- Single field observation (type=standard).
- Mosaic observation (type=standard).
- Ephemeris observation (type=ephemeris).
- Mosaic Ephemeris observation (type=ephemeris).

To satisfy these use cases, two types of `field_and_source_xds` were created: standard and ephemeris. The main difference is that the ephemeris type has a `FIELD_PHASE_OFFSET` data variable that is relative to the `SOURCE_POSITION/SOURCE_DIRECTION` data variable (contains the ephemerides and has a time axis), while the standard type has `FIELD_PHASE/DELAY/REFERENCE_CENTERS` and `SOURCE_POSITION` (has no time axis). The `SOURCE_POSITION/DIRECTION` is kept separate from the `FIELD_PHASE_OFFSET/CENTER` so that the intent `OBSERVE_TARGET#OFF_SOURCE` is supported and the ephemeris can be easily changed.

## Key Questions to Answer
### Schema Questions
- 1.1) Are there missing use cases?
- 1.2) Is all the information present needed for offline processing?
  - 1.2.1) Are there data variables we need to add (for example, the JPL Horizons data has additional information such as the North pole position angle, etc., see [CASA Ephemeris Data](https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data))?
  - 1.2.2) In some CASA ephemeris tables, there are table keywords such as `VS_CREATE`, `VS_DATE`, `VS_TYPE`, `VS_VERSION`, `MJD0`, `dMJD`, `earliest`, `latest`, `radii`, `meanradm`, `orb_per`, `rot_per`. Do we need any of these?
- 1.3) Is there a use case where the `FIELD_PHASE_CENTER` and `FIELD_DELAY_CENTER` would differ (i.e., do we need to store both)?
- 1.4) For interferometer observations, do we need to store the `FIELD_REFERENCE_CENTER` or can this be omitted (will still be present for Single dish)?
- 1.5) The ephemeris data is recorded in degrees, AU, and MJD. Should these be converted to radians, meters, and time (Unix)? Note that each data variable has measurement information attached to it. For example:
```Python
  frame: ICRS
  type: sky_coord
  units: ['deg', 'deg', 'AU']
```

- 1.6) For ephemeris observations, should we add the SOURCE_PROPER_MOTION if available?
- 1.7) Is the name `field_and_source_xds` sufficiently descriptive?
- 1.8) Should we also add the DOPPLER table information to the schema (if so, any idea where we can get an MSv2 with a DOPPLER table)?
- 1.9) Any naming suggestions or data layout?
- 1.10) Are the data variable descriptions in the schema spreadsheet correct?
- 1.11) What measures (https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1504318014) should we attach to each of the following data variables

  - NORTH_POLE_POSITION_ANGLE (quantity?)
  - NORTH_POLE_ANGULAR_DISTANCE (quantity?)
  - SUB_OBSERVER_DIRECTION (earth_location?)
  - SUB_SOLAR_POSITION (earth_location?)
  - HELIOCENTRIC_RADIAL_VELOCITY (quantity?)
  - OBSERVER_PHASE_ANGLE (quantity?)
- 1.12) Can NORTH_POLE_POSITION_ANGLE and NORTH_POLE_ANGULAR_DISTANCE be combined into a single data variable?

### XRADIO
2.1) After reviewing the XARRAY documentation and the descriptions of the data variables in the `field_and_source_xds` schema, do you find the XARRAY interface intuitive and easy to use?



[^1]: This is inhereted from MSv2 that only allows a single source per field [https://casacore.github.io/casacore-notes/229.pdf, p35], though a source can appear in more than one field.


# Environment instructions

It is recommended to use the conda environment manager to create a clean, self-contained runtime where xradio and all its dependencies can be installed:

```bash
conda create --name xradio python=3.11 --no-default-packages
conda activate xradio
```

Clone the repository, checkout the review branch and do a local install:

```bash
git clone https://github.com/casangi/xradio.git
git checkout 162-create-combined-field-source-and-ephemeris-dataset
cd xradio
pip install -e .
```

# ALMA Example

An ephemeris mosaic observation of the sun.

ALMA archive file downloaded: https://almascience.nrao.edu/dataPortal/2022.A.00001.S_uid___A002_X1003af4_X75a3.asdm.sdm.tar 

- Project: 2022.A.00001.S
- Member ous id (MOUS): uid://A001/X3571/X130
- Group ous id (GOUS): uid://A001/X3571/X131

CASA commands used to create the dataset:
```python
importasdm(asdm='uid___A002_X1003af4_X75a3.asdm.sdm',vis='uid___A002_X1003af4_X75a3.ms',asis='Ephemeris Antenna Station Receiver Source CalAtmosphere CalWVR',bdfflags=True,with_pointing_correction=True,convert_ephem2geo=True)

importasdm(asdm='22A-347.sb41163051.eb41573599.59642.62832791667.asdm',vis='22A-347.sb41163051.eb41573599.59642.62832791667.ms',asis='Ephemeris Antenna Station Receiver Source',with_pointing_correction=True,convert_ephem2geo=True)

mstransform(vis='ALMA_uid___A002_X1003af4_X75a3.split.ms',outputvis='ALMA_uid___A002_X1003af4_X75a3.split.avg.ms',createmms=False,timeaverage=True,timebin='2s',timespan='scan',reindex=True)

import numpy as np
tb.open('ALMA_uid___A002_X1003af4_X75a3.split.avg.ms::FLAG_CMD',nomodify=False)
tb.removerows(np.arange(tb.nrows())) # tb.removerows(np.arange(76401))
tb.flush()
tb.done()

tb.open('ALMA_uid___A002_X1003af4_X75a3.split.avg.ms::POINTING',nomodify=False)
tb.removerows(np.arange(tb.nrows()))   #tb.removerows(np.arange(5617115))
tb.flush()
tb.done()
```


## Download Data

In [1]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
from xradio.vis.read_processing_set import read_processing_set
import graphviper

graphviper.utils.data.download(file="ALMA_uid___A002_X1003af4_X75a3.split.avg.ms")

[2024-07-11 10:23:16,788]  WARNING      xradio:  File exists: /Users/jsteeb/Dropbox/graphviper/src/graphviper/utils/data/.dropbox 
[2024-07-11 10:23:16,795]     INFO      xradio:  Updating file metadata information ...  
 [2024-07-11 10:23:17,651]    DEBUG      xradio:  Initializing downloader with 1 threads.


 Download List                                
 ───────────────────────────────────────────── 
  ALMA_uid___A002_X1003af4_X75a3.split.avg.ms

[2024-07-11 10:23:17,685]     INFO      xradio:  File exists: ALMA_uid___A002_X1003af4_X75a3.split.avg.ms 


# Start Dask cluster 
Choose an approriate number of cores and memory_limit (this is per core).

In [2]:
from graphviper.dask.client import local_client

viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

[2024-07-11 10:23:17,786]     INFO      xradio:  Checking parameter values for client.local_client 
[2024-07-11 10:23:17,786]     INFO      xradio:  Module path: /Users/jsteeb/Dropbox/graphviper/ 
[2024-07-11 10:23:17,786]     INFO      xradio:  Searching /Users/jsteeb/Dropbox/graphviper/src/graphviper/config/ for configuration file, please wait ... 
[2024-07-11 10:23:17,787]    DEBUG      xradio:  Result: /Users/jsteeb/Dropbox/graphviper/src/graphviper/config
[2024-07-11 10:23:17,787]    DEBUG      xradio:  PARAMETER_CONFIG_PATH: /Users/jsteeb/Dropbox/graphviper/src/graphviper/config
[2024-07-11 10:23:17,787]    DEBUG      xradio:  /Users/jsteeb/Dropbox/graphviper/src/graphviper/config/client.param.json
[2024-07-11 10:23:17,791]  WARNING      client:  It is recommended that the local cache directory be set using the dask_local_dir parameter. 
[2024-07-11 10:23:18,461]     INFO      client:  Created client <MenrvaClient: 'tcp://127.0.0.1:61212' processes=4 threads=4, memory=14.90 GiB> 

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61212,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:61224,Total threads: 1
Dashboard: http://127.0.0.1:61228/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:61215,


# Convert dataset

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

in_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.ms"
out_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr"

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    partition_scheme="ddi_intent_field",
    parallel=True,
    overwrite=True,
    ephemeris_interpolate=True
)

[2024-07-11 10:23:18,502]     INFO      client:  Partition scheme that will be used: ['DATA_DESC_ID', 'INTENT', 'FIELD_ID'] 


[2024-07-11 10:23:19,019]     INFO      client:  Number of partitions: 96 
[2024-07-11 10:23:19,677]  WARNING    worker_0:  Unknown reference frame for direction measure, using verbatim: ICRS 
[2024-07-11 10:23:19,711]  WARNING    worker_3:  Unknown reference frame for direction measure, using verbatim: ICRS 
[2024-07-11 10:23:19,711]  WARNING    worker_1:  Unknown reference frame for direction measure, using verbatim: ICRS 
[2024-07-11 10:23:19,712]  WARNING    worker_2:  Unknown reference frame for direction measure, using verbatim: ICRS 
[2024-07-11 10:23:19,988]  WARNING    worker_0:  Unknown reference frame for direction measure, using verbatim: ICRS 
[2024-07-11 10:23:20,012]  WARNING    worker_1:  Unknown reference frame for direction measure, using verbatim: ICRS 
[2024-07-11 10:23:20,019]  WARNING    worker_3:  Unknown reference frame for direction measure, using verbatim: ICRS 
[2024-07-11 10:23:20,043]  WARNING    worker_2:  Unknown reference frame for direction measure, usi

# Inspect Processing Set

In [4]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option("display.max_rows", 1000)

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency,shape,field_coords
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_52,1,OBSERVE_TARGET#ON_SOURCE,18,Sun_10,1.845500e+11,1.905500e+11,"(9, 51, 4, 1)",Ephemeris
1,ALMA_uid___A002_X1003af4_X75a3.split.avg_55,1,OBSERVE_TARGET#ON_SOURCE,21,Sun_10,1.845500e+11,1.905500e+11,"(9, 51, 4, 1)",Ephemeris
2,ALMA_uid___A002_X1003af4_X75a3.split.avg_63,1,OBSERVE_TARGET#ON_SOURCE,29,Sun_10,1.845500e+11,1.905500e+11,"(6, 51, 4, 1)",Ephemeris
3,ALMA_uid___A002_X1003af4_X75a3.split.avg_90,2,OBSERVE_TARGET#ON_SOURCE,24,Sun_10,2.299609e+11,2.300547e+11,"(9, 1326, 7, 2)",Ephemeris
4,ALMA_uid___A002_X1003af4_X75a3.split.avg_64,2,"CALIBRATE_ATMOSPHERE#OFF_SOURCE,CALIBRATE_WVR#...",0,Sun_10,2.299609e+11,2.300547e+11,"(2, 1326, 7, 2)",Ephemeris
5,ALMA_uid___A002_X1003af4_X75a3.split.avg_30,0,OBSERVE_TARGET#ON_SOURCE,28,Sun_10,2.480000e+11,2.480000e+11,"(8, 51, 1, 2)",Ephemeris
6,ALMA_uid___A002_X1003af4_X75a3.split.avg_37,1,OBSERVE_TARGET#ON_SOURCE,3,Sun_10,1.845500e+11,1.905500e+11,"(9, 51, 4, 1)",Ephemeris
7,ALMA_uid___A002_X1003af4_X75a3.split.avg_39,1,OBSERVE_TARGET#ON_SOURCE,5,Sun_10,1.845500e+11,1.905500e+11,"(9, 51, 4, 1)",Ephemeris
8,ALMA_uid___A002_X1003af4_X75a3.split.avg_65,2,"CALIBRATE_ATMOSPHERE#AMBIENT,CALIBRATE_WVR#AMB...",0,Sun_10,2.299609e+11,2.300547e+11,"(2, 1326, 7, 2)",Ephemeris
9,ALMA_uid___A002_X1003af4_X75a3.split.avg_91,2,OBSERVE_TARGET#ON_SOURCE,25,Sun_10,2.299609e+11,2.300547e+11,"(9, 1326, 7, 2)",Ephemeris


In [5]:
ps['ALMA_uid___A002_X1003af4_X75a3.split.avg_52'].attrs['partition_info']

{'field_id': 18,
 'field_name': 'Sun_10',
 'intent': 'OBSERVE_TARGET#ON_SOURCE',
 'polarization_setup': ['XX'],
 'source_id': 0,
 'source_name': 'Sun_10',
 'spectral_window_id': 1,
 'spectral_window_name': 'WVR#NOMINAL',
 'taql': 'WHERE (DATA_DESC_ID IN [1]) AND(STATE_ID IN [36,63,90]) AND(FIELD_ID IN [18]) AND(SCAN_NUMBER IN [8]) AND(STATE_ID IN [36,63,90]) '}

In [6]:
ps['ALMA_uid___A002_X1003af4_X75a3.split.avg_52'].VISIBILITY.attrs["field_and_source_xds"].is_ephemeris

True

# Inspect field_and_source_xds: Standard Use case (non-ephemeris)

In [7]:
standard_field_and_source_xds = ps[
    "ALMA_uid___A002_X1003af4_X75a3.split.avg_67"
].VISIBILITY.field_and_source_xds.load()  # Load the data into memory
standard_field_and_source_xds

<xarray.Dataset> Size: 160B
Dimensions:                 (sky_dir_label: 2, line_name: 1)
Coordinates:
  * line_name               (line_name) <U22 88B 'Single_Continuum(ID=0)'
  * sky_dir_label           (sky_dir_label) <U3 24B 'ra' 'dec'
Data variables:
    FIELD_PHASE_CENTER      (sky_dir_label) float64 16B -2.579 -0.1374
    LINE_REST_FREQUENCY     (line_name) float64 8B 2.3e+11
    LINE_SYSTEMIC_VELOCITY  (line_name) float64 8B 0.0
    SOURCE_DIRECTION        (sky_dir_label) float64 16B -2.579 -0.1374
Attributes:
    code:          none
    field_code:    none
    field_name:    J1408-0752
    is_ephemeris:  False
    source_id:     2
    source_name:   J1408-0752

In [8]:
standard_field_and_source_xds  # How to access field_and_source_xds.

<xarray.Dataset> Size: 160B
Dimensions:                 (sky_dir_label: 2, line_name: 1)
Coordinates:
  * line_name               (line_name) <U22 88B 'Single_Continuum(ID=0)'
  * sky_dir_label           (sky_dir_label) <U3 24B 'ra' 'dec'
Data variables:
    FIELD_PHASE_CENTER      (sky_dir_label) float64 16B -2.579 -0.1374
    LINE_REST_FREQUENCY     (line_name) float64 8B 2.3e+11
    LINE_SYSTEMIC_VELOCITY  (line_name) float64 8B 0.0
    SOURCE_DIRECTION        (sky_dir_label) float64 16B -2.579 -0.1374
Attributes:
    code:          none
    field_code:    none
    field_name:    J1408-0752
    is_ephemeris:  False
    source_id:     2
    source_name:   J1408-0752

In [9]:
standard_field_and_source_xds.FIELD_PHASE_CENTER  # How to access field_and_source_xds datavariables. standard_field_and_source_xds['FIELD_PHASE_CENTER'] can also be used.

<xarray.DataArray 'FIELD_PHASE_CENTER' (sky_dir_label: 2)> Size: 16B
array([-2.57897986, -0.13742852])
Coordinates:
  * sky_dir_label  (sky_dir_label) <U3 24B 'ra' 'dec'
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad']

In [10]:
standard_field_and_source_xds.FIELD_PHASE_CENTER.attrs  # How to access field_and_source_xds datavariables measures information stored in the attributes.

{'frame': 'ICRS', 'type': 'sky_coord', 'units': ['rad', 'rad']}

# Inspect field_and_source_xds: Ephemeris Use case (Mosaic) with line

In [11]:
ephemeris_field_and_source_xds = ps[
    "ALMA_uid___A002_X1003af4_X75a3.split.avg_52"
].VISIBILITY.field_and_source_xds.load()  # Load the data into memory
ephemeris_field_and_source_xds

<xarray.Dataset> Size: 1kB
Dimensions:                       (time: 9, sky_pos_label: 3,
                                   ellipsoid_pos_label: 3, sky_dir_label: 2)
Coordinates:
  * ellipsoid_pos_label           (ellipsoid_pos_label) <U4 48B 'lon' ... 'dist'
  * sky_dir_label                 (sky_dir_label) <U3 24B 'ra' 'dec'
  * sky_pos_label                 (sky_pos_label) <U4 48B 'ra' 'dec' 'dist'
  * time                          (time) float64 72B 1.667e+09 ... 1.667e+09
Data variables:
    FIELD_PHASE_CENTER            (time, sky_pos_label) float64 216B -2.624 ....
    HELIOCENTRIC_RADIAL_VELOCITY  (time) float64 72B 0.0 0.0 0.0 ... 0.0 0.0 0.0
    NORTH_POLE_ANGULAR_DISTANCE   (time) float64 72B 544.0 544.1 ... 548.8 548.8
    NORTH_POLE_POSITION_ANGLE     (time) float64 72B 14.54 14.54 ... 14.66 14.66
    OBSERVATION_POSITION          (ellipsoid_pos_label) float64 24B 0.0 0.0 0.0
    OBSERVER_PHASE_ANGLE          (time) float64 72B 0.0 0.0 0.0 ... 0.0 0.0 0.0
    SOURCE_POSITION               (time, sky_pos_label) float64 216B 3.659 .....
    SOURCE_RADIAL_VELOCITY        (time) float64 72B -204.7 -204.7 ... -202.3
    SUB_OBSERVER_POSITION         (time, ellipsoid_pos_label) float64 216B 3....
Attributes:
    code:          none
    field_code:    none
    field_name:    Sun_10
    is_ephemeris:  True
    source_id:     0
    source_name:   Sun_10

In [12]:
ephemeris_field_and_source_xds.FIELD_PHASE_CENTER

<xarray.DataArray 'FIELD_PHASE_CENTER' (time: 9, sky_pos_label: 3)> Size: 216B
array([[-2.62424049e+00, -2.11160026e-01,  1.48765920e+11],
       [-2.62424004e+00, -2.11160187e-01,  1.48765919e+11],
       [-2.62423971e+00, -2.11160307e-01,  1.48765918e+11],
       [-2.62419908e+00, -2.11174953e-01,  1.48765819e+11],
       [-2.62419864e+00, -2.11175114e-01,  1.48765818e+11],
       [-2.62419831e+00, -2.11175234e-01,  1.48765817e+11],
       [-2.62415790e+00, -2.11189801e-01,  1.48765719e+11],
       [-2.62415746e+00, -2.11189961e-01,  1.48765718e+11],
       [-2.62415712e+00, -2.11190081e-01,  1.48765717e+11]])
Coordinates:
  * sky_pos_label  (sky_pos_label) <U4 48B 'ra' 'dec' 'dist'
  * time           (time) float64 72B 1.667e+09 1.667e+09 ... 1.667e+09
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad', 'm']

In [13]:
ephemeris_field_and_source_xds.SOURCE_POSITION

<xarray.DataArray 'SOURCE_POSITION' (time: 9, sky_pos_label: 3)> Size: 216B
array([[ 3.65894482e+00, -2.11185210e-01,  1.48765920e+11],
       [ 3.65894526e+00, -2.11185371e-01,  1.48765919e+11],
       [ 3.65894560e+00, -2.11185491e-01,  1.48765918e+11],
       [ 3.65898622e+00, -2.11200138e-01,  1.48765819e+11],
       [ 3.65898667e+00, -2.11200298e-01,  1.48765818e+11],
       [ 3.65898700e+00, -2.11200419e-01,  1.48765817e+11],
       [ 3.65902740e+00, -2.11214985e-01,  1.48765719e+11],
       [ 3.65902785e+00, -2.11215145e-01,  1.48765718e+11],
       [ 3.65902818e+00, -2.11215266e-01,  1.48765717e+11]])
Coordinates:
  * sky_pos_label  (sky_pos_label) <U4 48B 'ra' 'dec' 'dist'
  * time           (time) float64 72B 1.667e+09 1.667e+09 ... 1.667e+09
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad', 'm']

In [14]:
ephemeris_field_and_source_xds.SOURCE_POSITION.sel(sky_pos_label="dec")

<xarray.DataArray 'SOURCE_POSITION' (time: 9)> Size: 72B
array([-0.21118521, -0.21118537, -0.21118549, -0.21120014, -0.2112003 ,
       -0.21120042, -0.21121498, -0.21121515, -0.21121527])
Coordinates:
    sky_pos_label  <U4 16B 'dec'
  * time           (time) float64 72B 1.667e+09 1.667e+09 ... 1.667e+09
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad', 'm']

In [15]:
ps[
    "ALMA_uid___A002_X1003af4_X75a3.split.avg_52"
]

<xarray.Dataset> Size: 43kB
Dimensions:                     (time: 9, baseline_id: 51, frequency: 4,
                                 polarization: 1, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int32 204B dask.array<chunksize=(51,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int32 204B dask.array<chunksize=(51,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 408B 0 1 2 3 ... 47 48 49 50
  * frequency                   (frequency) float64 32B 1.846e+11 ... 1.906e+11
  * polarization                (polarization) <U2 8B 'XX'
  * time                        (time) float64 72B 1.667e+09 ... 1.667e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 4kB dask.array<chunksize=(9, 51), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 2kB dask.array<chunksize=(9, 51, 4, 1), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 4kB dask.array<chunksize=(9, 51), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 11kB dask.array<chunksize=(9, 51, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 15kB dask.array<chunksize=(9, 51, 4, 1), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 7kB dask.array<chunksize=(9, 51, 4, 1), meta=np.ndarray>
Attributes:
    data_groups:     {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'V...
    ddi:             1
    intent:          OBSERVE_TARGET#ON_SOURCE
    partition_info:  {'field_id': 18, 'field_name': 'Sun_10', 'intent': 'OBSE...
    pointing_xds:    <xarray.Dataset> Size: 0B\nDimensions:  ()\nData variabl...
    weather_xds:     <xarray.Dataset> Size: 138kB\nDimensions:         (stati...
    antenna_xds:     <xarray.Dataset> Size: 9kB\nDimensions:        (antenna_...